# Add Monte Carlo Dropout to PyTorch LSTM (Phase 2)

This notebook adds Monte Carlo Dropout uncertainty quantification to the PyTorch LSTM model.

**Why PyTorch for MC Dropout?**
- Much simpler: Just use `model.train()` during inference
- No need for special wrappers or functional API
- Faster on Apple Silicon with MPS

**Requirements:** 
- Run `03_train_lstm_pytorch.ipynb` first to train the base LSTM model
- This notebook loads the saved model and adds uncertainty via MC Dropout


In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import joblib
import json
import time

# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Add project root to path
project_root = Path("/Users/siddhantaggarwal/Desktop/Battery_RUL").resolve()
sys.path.append(str(project_root))

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Set device
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("✅ Using MPS (Apple Silicon GPU acceleration)")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("✅ Using CUDA (NVIDIA GPU)")
else:
    device = torch.device("cpu")
    print("⚠️ Using CPU")

print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}")
print("✅ PyTorch ready!")


In [ ]:
# Load dataset
processed_dir = project_root / "data" / "processed"
df = pd.read_parquet(processed_dir / "rul_features_with_emd.parquet")

df_clean = df[df['RUL'].notna()].copy()
print(f"Dataset: {len(df_clean)} rows")

# Prepare features (same as training)
exclude_cols = [
    'battery_id', 'filename', 'type', 'start_time', 'test_id', 'uid',
    'split', 'cycle_index', 'EOL_cycle', 'RUL', 'SOH', 'Capacity', 
    'Re', 'Rct', 'ambient_temperature'
]

feature_cols = [c for c in df_clean.columns if c not in exclude_cols]
print(f"Features: {len(feature_cols)}")

# Split by battery
test_idx = df_clean['split'] == 'test'
df_test = df_clean[test_idx].copy()
print(f"Test set: {len(df_test)} rows")


In [ ]:
# Load model architecture and weights
models_dir = project_root / "results" / "models"

# Load model info
with open(models_dir / "lstm_pytorch_model_info.json", 'r') as f:
    model_info = json.load(f)

print("Model architecture:")
for key, value in model_info.items():
    print(f"  {key}: {value}")

# Define model class (same as training)
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size1=64, hidden_size2=32, num_layers=1, dropout=0.2):
        super(LSTMModel, self).__init__()
        
        self.lstm1 = nn.LSTM(input_size, hidden_size1, num_layers, 
                            batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.lstm2 = nn.LSTM(hidden_size1, hidden_size2, num_layers,
                            batch_first=True, dropout=dropout if num_layers > 1 else 0)
        
        self.fc1 = nn.Linear(hidden_size2, 16)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(16, 1)
    
    def forward(self, x):
        out, _ = self.lstm1(x)
        out, _ = self.lstm2(out)
        out = out[:, -1, :]
        out = self.fc1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        return out.squeeze()

# Create and load model
model = LSTMModel(
    input_size=model_info['input_size'],
    hidden_size1=model_info['hidden_size1'],
    hidden_size2=model_info['hidden_size2'],
    dropout=model_info['dropout']
).to(device)

model.load_state_dict(torch.load(models_dir / "lstm_pytorch_point_model.pth"))
print("✅ Model loaded!")

# Load scaler
scaler = joblib.load(models_dir / "lstm_pytorch_scaler.pkl")
print("✅ Scaler loaded!")


In [ ]:
# Create sequences for test set
sequence_length = model_info['sequence_length']

def create_sequences(df_subset, feature_cols, target_col, seq_len=sequence_length):
    """Create sequences for LSTM."""
    sequences = []
    targets = []
    
    df_filled = df_subset[feature_cols + [target_col, 'battery_id', 'cycle_index']].fillna(0)
    
    for battery_id in df_subset['battery_id'].unique():
        battery_data = df_filled[df_filled['battery_id'] == battery_id].sort_values('cycle_index')
        
        features = battery_data[feature_cols].values.astype(np.float32)
        target = battery_data[target_col].values.astype(np.float32)
        
        if len(features) >= seq_len:
            n_seqs = len(features) - seq_len
            for i in range(n_seqs):
                sequences.append(features[i:i+seq_len])
                targets.append(target[i+seq_len])
    
    return np.array(sequences, dtype=np.float32), np.array(targets, dtype=np.float32)

# Create test sequences
X_test_seq, y_test_seq = create_sequences(df_test, feature_cols, 'RUL', sequence_length)

# Normalize
n_samples, n_timesteps, n_features = X_test_seq.shape
X_test_reshaped = X_test_seq.reshape(-1, n_features)
X_test_scaled = scaler.transform(X_test_reshaped)
X_test_seq = X_test_scaled.reshape(n_samples, n_timesteps, n_features)

print(f"Test sequences: {X_test_seq.shape}")


In [ ]:
# Monte Carlo Dropout Inference
# In PyTorch, this is SUPER simple: just use model.train() during inference!

n_samples_mc = 100  # Number of Monte Carlo samples

print(f"Running {n_samples_mc} Monte Carlo forward passes with dropout enabled...")
print("(This is much simpler in PyTorch - just set model.train()!)")

# Enable dropout during inference
model.train()  # This enables dropout layers

# Collect predictions
mc_predictions = []

start_time = time.time()
with torch.no_grad():  # No gradient computation needed
    X_test_tensor = torch.FloatTensor(X_test_seq).to(device)
    
    for i in range(n_samples_mc):
        pred = model(X_test_tensor)
        mc_predictions.append(pred.cpu().numpy())
        
        if (i + 1) % 20 == 0:
            print(f"  Completed {i+1}/{n_samples_mc} samples...")

elapsed = time.time() - start_time
print(f"✅ Completed in {elapsed:.2f} seconds ({elapsed/n_samples_mc*1000:.1f} ms per sample)")

# Stack predictions: shape (n_samples_mc, n_test_samples)
mc_predictions = np.array(mc_predictions).T  # Shape: (n_test_samples, n_samples_mc)

print(f"MC predictions shape: {mc_predictions.shape} (samples, n_passes)")

# Calculate statistics
pred_mean = np.mean(mc_predictions, axis=1)
pred_std = np.std(mc_predictions, axis=1)

# Calculate percentiles
pred_q05 = np.percentile(mc_predictions, 5, axis=1)
pred_q25 = np.percentile(mc_predictions, 25, axis=1)
pred_q75 = np.percentile(mc_predictions, 75, axis=1)
pred_q95 = np.percentile(mc_predictions, 95, axis=1)

print(f"\n✅ Uncertainty statistics calculated:")
print(f"   Mean prediction range: [{pred_mean.min():.1f}, {pred_mean.max():.1f}]")
print(f"   Uncertainty (std) range: [{pred_std.min():.2f}, {pred_std.max():.2f}]")


In [ ]:
# Evaluate uncertainty calibration
# Check if prediction intervals actually cover the true values

# 90% prediction interval (5th to 95th percentile)
coverage_90 = np.mean((y_test_seq >= pred_q05) & (y_test_seq <= pred_q95))
interval_width_90 = np.mean(pred_q95 - pred_q05)

# 50% prediction interval (25th to 75th percentile)
coverage_50 = np.mean((y_test_seq >= pred_q25) & (y_test_seq <= pred_q75))
interval_width_50 = np.mean(pred_q75 - pred_q25)

print("Uncertainty Calibration:")
print(f"  90% Prediction Interval:")
print(f"    Coverage: {coverage_90*100:.1f}% (target: 90%)")
print(f"    Average width: {interval_width_90:.2f} cycles")
print(f"  50% Prediction Interval:")
print(f"    Coverage: {coverage_50*100:.1f}% (target: 50%)")
print(f"    Average width: {interval_width_50:.2f} cycles")

# Point prediction metrics (using mean of MC samples)
def calculate_metrics(y_true, y_pred, name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / (np.abs(y_true) + 1e-6))) * 100
    
    print(f"\n{name} Metrics (MC Dropout mean):")
    print(f"  MAE:  {mae:.2f} cycles")
    print(f"  RMSE: {rmse:.2f} cycles")
    print(f"  R²:   {r2:.3f}")
    print(f"  MAPE: {mape:.2f}%")
    return {'mae': mae, 'rmse': rmse, 'r2': r2, 'mape': mape}

mc_metrics = calculate_metrics(y_test_seq, pred_mean, "Test (MC Dropout)")


In [ ]:
# Visualize predictions with uncertainty intervals
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: Predictions vs Actual with uncertainty
sample_idx = np.random.choice(len(y_test_seq), min(200, len(y_test_seq)), replace=False)
sorted_idx = np.argsort(y_test_seq[sample_idx])

axes[0].scatter(range(len(sorted_idx)), pred_mean[sample_idx][sorted_idx], 
                alpha=0.6, s=20, label='Mean Prediction', zorder=3)
axes[0].fill_between(range(len(sorted_idx)), 
                     pred_q05[sample_idx][sorted_idx],
                     pred_q95[sample_idx][sorted_idx],
                     alpha=0.3, label='90% Prediction Interval', color='blue', zorder=1)
axes[0].fill_between(range(len(sorted_idx)), 
                     pred_q25[sample_idx][sorted_idx],
                     pred_q75[sample_idx][sorted_idx],
                     alpha=0.5, label='50% Prediction Interval', color='blue', zorder=2)
axes[0].plot(range(len(sorted_idx)), y_test_seq[sample_idx][sorted_idx], 
             'r-', linewidth=2, label='Actual', zorder=4)
axes[0].set_xlabel('Sample Index (sorted by actual RUL)')
axes[0].set_ylabel('RUL (cycles)')
axes[0].set_title('LSTM Predictions with MC Dropout Uncertainty (Test Set)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Uncertainty vs Error
abs_error = np.abs(y_test_seq - pred_mean)
axes[1].scatter(pred_std, abs_error, alpha=0.5)
axes[1].set_xlabel('Predicted Uncertainty (std)')
axes[1].set_ylabel('Absolute Error')
axes[1].set_title('Uncertainty vs Prediction Error')
axes[1].grid(True, alpha=0.3)

# Add correlation
correlation = np.corrcoef(pred_std, abs_error)[0, 1]
axes[1].text(0.05, 0.95, f'Correlation: {correlation:.3f}', 
             transform=axes[1].transAxes, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()


In [ ]:
# Save results with uncertainty
results = pd.DataFrame({
    'actual_rul': y_test_seq,
    'predicted_rul_mean': pred_mean,
    'predicted_rul_std': pred_std,
    'predicted_rul_q05': pred_q05,
    'predicted_rul_q25': pred_q25,
    'predicted_rul_q75': pred_q75,
    'predicted_rul_q95': pred_q95
})

results_path = models_dir / "lstm_pytorch_predictions_mc_dropout.csv"
results.to_csv(results_path, index=False)
print(f"✅ Saved predictions with uncertainty: {results_path}")

# Save metrics
metrics = pd.DataFrame({
    'metric': ['MAE', 'RMSE', 'R²', 'MAPE', 'Coverage_90%', 'Coverage_50%', 'Width_90%', 'Width_50%'],
    'value': [
        mc_metrics['mae'], 
        mc_metrics['rmse'], 
        mc_metrics['r2'], 
        mc_metrics['mape'],
        coverage_90,
        coverage_50,
        interval_width_90,
        interval_width_50
    ]
})

metrics_path = models_dir / "lstm_pytorch_metrics_mc_dropout.csv"
metrics.to_csv(metrics_path, index=False)
print(f"✅ Saved metrics: {metrics_path}")

print(f"\n📊 Summary:")
print(f"   Test MAE: {mc_metrics['mae']:.2f} cycles")
print(f"   Test RMSE: {mc_metrics['rmse']:.2f} cycles")
print(f"   Test R²: {mc_metrics['r2']:.3f}")
print(f"   90% Coverage: {coverage_90*100:.1f}%")
print(f"   50% Coverage: {coverage_50*100:.1f}%")
print(f"\n✅ Phase 2 Complete: Monte Carlo Dropout uncertainty quantification added!")
print(f"   Uncertainty intervals saved and ready for dashboard!")
